Data we have:
1. Protein sequences in string format
2. Protein PDB sequences in string format<br>

We need to use these to predict druggability of the protein

In [1]:

import json

with open("../drugbank/protein_drugbank.json") as f:
    drugbank_info = json.load(f)
    druggable_proteins = list(drugbank_info.keys())

print("Number of druggable proteins:", len(druggable_proteins))
# encoding protein
approved_druggable_proteins = []
for protein in druggable_proteins:
    for k,v in drugbank_info[protein].items():
        if "approved" in v[1]:
            approved_druggable_proteins.append(protein)
            break

print("Number of approved druggable proteins:", len(approved_druggable_proteins))

Number of druggable proteins: 3345
Number of approved druggable proteins: 2652


In [2]:
with open("pdb_sequences.json") as f:
    pdb_sequences = json.load(f)

proteins = list(pdb_sequences.keys())
print("Total number of proteins:", len(proteins))

Total number of proteins: 20434


In [3]:
# From the pdb_sequences, find number of proteins, which have pdb_seq info
pdb_seq_proteins = []
for protein in proteins:
    if pdb_sequences[protein]["pdb_seq"] != "N"*len(pdb_sequences[protein]["pdb_seq"]):
        pdb_seq_proteins.append(protein)

print("Number of proteins with pdb_seq:", len(pdb_seq_proteins))

Number of proteins with pdb_seq: 7347


In [4]:
# Find number of druggable and non druggable proteins in this pdb_seq_proteins
# We consider only those with approved drugs as druggable, rest all are non druggable
druggable_pdb_seq_proteins = []
for protein in pdb_seq_proteins:
    if protein in approved_druggable_proteins:
        druggable_pdb_seq_proteins.append(protein)

print("Number of druggable proteins with pdb_seq:", len(druggable_pdb_seq_proteins))

Number of druggable proteins with pdb_seq: 1755


As an initial attempt, we choose features as
1. Amino Acid Composition
2. Beta-Helix-Turn Composition

Ablation Study<br>
a. Only Property 1<br>
b. Property 1 + Property 2

In [5]:
# check unique characters in sequnces of proteins+ their frequency
unique_chars = {}
for protein in pdb_seq_proteins:
    for char in pdb_sequences[protein]["seq"]:
        if char in unique_chars:
            unique_chars[char] += 1
        else:
            unique_chars[char] = 1
unique_chars, len(unique_chars)

({'M': 97991,
  'G': 293339,
  'A': 309713,
  'T': 240378,
  'Q': 213298,
  'L': 430478,
  'V': 273214,
  'F': 159932,
  'E': 326578,
  'K': 267489,
  'I': 200176,
  'R': 249418,
  'H': 110857,
  'S': 363990,
  'N': 169177,
  'P': 278523,
  'Y': 121627,
  'C': 92974,
  'D': 227557,
  'W': 52558},
 20)

In [6]:
def get_amino_acid_composition(seq):
    amino_acids = unique_chars.keys()
    composition = {}
    for aa in amino_acids:
        composition[aa] = seq.count(aa)/len(seq)
    return list(composition.values())

def get_pdb_composition(pdb_seq):
    pdb_components = ["N", "H", "E", "T"]
    composition = {}
    for aa in pdb_components:
        composition[aa] = pdb_seq.count(aa)/len(pdb_seq)
    return list(composition.values())

### Amino Acid Composition

In [19]:
# Collect features X and labels Y
# X being amino acid composition of protein sequence and Y being druggable(1) or not(0)
X = []
Y = []
for protein in pdb_seq_proteins:
    X.append(get_amino_acid_composition(pdb_sequences[protein]["seq"]))
    if protein in druggable_pdb_seq_proteins:
        Y.append(1)
    else:
        Y.append(0)

In [20]:
import numpy as np
X = np.array(X)
Y = np.array(Y)
X.shape, Y.shape

((7347, 20), (7347,))

In [24]:
# separate druggable and non druggable X
X_druggable = X[Y==1]
X_non_druggable = X[Y==0]
X_druggable.shape, X_non_druggable.shape

((1755, 20), (5592, 20))

In [27]:
# Let's choose 1000 druggable and 1000 non druggable proteins for training
# Shuffle the data using np.random.shuffle
np.random.seed(0)
np.random.shuffle(X_druggable)
np.random.shuffle(X_non_druggable)

X_train = np.concatenate((X_druggable[:1000], X_non_druggable[:1000]), axis=0)
Y_train = np.concatenate((np.ones(1000), np.zeros(1000)), axis=0)

X_train.shape, Y_train.shape

((2000, 20), (2000,))

In [33]:
# Concatenate X_train and Y_train
data = np.concatenate((X_train, Y_train.reshape(-1,1)), axis=1)
data.shape

# random shuffle the data
np.random.shuffle(data)

# Break the data back into X_train and Y_train
X_train = data[:, :-1]
Y_train = data[:, -1]

X_train.shape, Y_train.shape

((2000, 20), (2000,))

In [64]:
# Train a logistic regression model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty="elasticnet", warm_start=True, solver="saga", l1_ratio=0.5, max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(l1_ratio=0.5, max_iter=1000, penalty='elasticnet',
                   solver='saga', warm_start=True)

In [72]:
def print_stats(model, X, Y):
    Y_pred = model.predict(X)
    TP = np.sum((Y==1) & (Y_pred==1))
    FN = np.sum((Y==1) & (Y_pred==0))
    FP = np.sum((Y==0) & (Y_pred==1))
    TN = np.sum((Y==0) & (Y_pred==0))

    print("TP:", TP, "FN:", FN, "FP:", FP, "TN:", TN)

In [65]:
# Give training stats on the model
# TP, TN, FP, FN
Y_pred = model.predict(X_train)
TP = np.sum((Y_train==1) & (Y_pred==1))
FP = np.sum((Y_train==0) & (Y_pred==1))
TN = np.sum((Y_train==0) & (Y_pred==0))
FN = np.sum((Y_train==1) & (Y_pred==0))
TP, FP, TN, FN # Not able to predict any druggable protein

(691, 399, 601, 309)

In [69]:
X_test = np.concatenate((X_druggable[1000:], X_non_druggable[1000:]), axis=0)
Y_test = np.concatenate((np.ones(len(X_druggable[1000:])), np.zeros(len(X_non_druggable[1000:]))), axis=0)

Y_pred = model.predict(X_test)
TP = np.sum((Y_test==1) & (Y_pred==1))
FP = np.sum((Y_test==0) & (Y_pred==1))
TN = np.sum((Y_test==0) & (Y_pred==0))
FN = np.sum((Y_test==1) & (Y_pred==0))
TP, FP, TN, FN # Not able to predict any druggable protein

(523, 1801, 2791, 232)

In [90]:
# Let's use a neural network model
from sklearn.neural_network import MLPClassifier

np.random.seed(0)
model = MLPClassifier(hidden_layer_sizes=(100, 50, 50, 10), max_iter=1000, warm_start=True, solver="adam", activation="relu", learning_rate="adaptive", verbose=True)
model.fit(X_train, Y_train)

Iteration 1, loss = 0.69380108
Iteration 2, loss = 0.69281519
Iteration 3, loss = 0.69233447
Iteration 4, loss = 0.69166630
Iteration 5, loss = 0.69071366
Iteration 6, loss = 0.68997465
Iteration 7, loss = 0.68851109
Iteration 8, loss = 0.68628043
Iteration 9, loss = 0.68329588
Iteration 10, loss = 0.68004182
Iteration 11, loss = 0.67474920
Iteration 12, loss = 0.66873077
Iteration 13, loss = 0.66164489
Iteration 14, loss = 0.65218190
Iteration 15, loss = 0.64387998
Iteration 16, loss = 0.63931488
Iteration 17, loss = 0.63534236
Iteration 18, loss = 0.63147927
Iteration 19, loss = 0.62911596
Iteration 20, loss = 0.62936106
Iteration 21, loss = 0.62775759
Iteration 22, loss = 0.63137808
Iteration 23, loss = 0.62688726
Iteration 24, loss = 0.62990913
Iteration 25, loss = 0.62835263
Iteration 26, loss = 0.63015376
Iteration 27, loss = 0.62690945
Iteration 28, loss = 0.62606203
Iteration 29, loss = 0.62634701
Iteration 30, loss = 0.62496641
Iteration 31, loss = 0.62665972
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(100, 50, 50, 10), learning_rate='adaptive',
              max_iter=1000, verbose=True, warm_start=True)

In [91]:
print_stats(model, X_train, Y_train)

TP: 769 FN: 231 FP: 331 TN: 669


In [92]:
print_stats(model, X_test, Y_test)

TP: 526 FN: 229 FP: 1874 TN: 2718


In [12]:
print(769/1000, 669/1000) # Training accuracy in positive and negative classes
print(526/(526+229), 2718/(1874+2718)) # Testing accuracy in positive and negative classes

0.769 0.669
0.6966887417218544 0.5918989547038328


### Amino Acid + PDB Composition

In [93]:
X = []
Y = []
for protein in pdb_seq_proteins:
    X.append(get_amino_acid_composition(pdb_sequences[protein]["seq"]) + get_pdb_composition(pdb_sequences[protein]["pdb_seq"]))
    if protein in druggable_pdb_seq_proteins:
        Y.append(1)
    else:
        Y.append(0)

X = np.array(X)
Y = np.array(Y)
X.shape, Y.shape

((7347, 24), (7347,))

In [94]:
X_druggable = X[Y==1]
X_non_druggable = X[Y==0]

np.random.seed(0)
np.random.shuffle(X_druggable)
np.random.shuffle(X_non_druggable)

X_train = np.concatenate((X_druggable[:1000], X_non_druggable[:1000]), axis=0)
Y_train = np.concatenate((np.ones(1000), np.zeros(1000)), axis=0)

data = np.concatenate((X_train, Y_train.reshape(-1,1)), axis=1)
np.random.shuffle(data)

X_train = data[:, :-1]
Y_train = data[:, -1]

X_test = np.concatenate((X_druggable[1000:], X_non_druggable[1000:]), axis=0)
Y_test = np.concatenate((np.ones(len(X_druggable[1000:])), np.zeros(len(X_non_druggable[1000:]))), axis=0)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((2000, 24), (2000,), (5347, 24), (5347,))

In [101]:
# Logistic regression model
np.random.seed(0)
model = LogisticRegression(penalty="elasticnet", warm_start=True, solver="saga", l1_ratio=0.5, max_iter=1000)
model.fit(X_train, Y_train)

print_stats(model, X_train, Y_train)
print_stats(model, X_test, Y_test)

TP: 676 FN: 324 FP: 423 TN: 577
TP: 526 FN: 229 FP: 1872 TN: 2720


In [102]:
# Neural network model
np.random.seed(0)
model = MLPClassifier(hidden_layer_sizes=(100, 50, 50, 10), max_iter=1000, warm_start=True, solver="adam", activation="relu", learning_rate="adaptive", verbose=True)
model.fit(X_train, Y_train)

print_stats(model, X_train, Y_train)
print_stats(model, X_test, Y_test)

Iteration 1, loss = 0.71986156
Iteration 2, loss = 0.68871544
Iteration 3, loss = 0.68301885
Iteration 4, loss = 0.67471392
Iteration 5, loss = 0.66269385
Iteration 6, loss = 0.65569090
Iteration 7, loss = 0.65155381
Iteration 8, loss = 0.65076291
Iteration 9, loss = 0.64876795
Iteration 10, loss = 0.64774108
Iteration 11, loss = 0.64635131
Iteration 12, loss = 0.64460503
Iteration 13, loss = 0.64381048
Iteration 14, loss = 0.64451909
Iteration 15, loss = 0.64324714
Iteration 16, loss = 0.63942918
Iteration 17, loss = 0.64129227
Iteration 18, loss = 0.63827998
Iteration 19, loss = 0.63748101
Iteration 20, loss = 0.63630674
Iteration 21, loss = 0.63414008
Iteration 22, loss = 0.63139745
Iteration 23, loss = 0.63114305
Iteration 24, loss = 0.62740440
Iteration 25, loss = 0.62601836
Iteration 26, loss = 0.62556172
Iteration 27, loss = 0.62191620
Iteration 28, loss = 0.62192704
Iteration 29, loss = 0.61953621
Iteration 30, loss = 0.61863272
Iteration 31, loss = 0.62181570
Iteration 32, los

In [13]:
print(761/1000, 793/1000) # train accuracy in positive and negative classes
print(474/(474+281), 3084/(3084+1508)) # test accuracy in positive and negative classes

0.761 0.793
0.6278145695364239 0.671602787456446


Performance is still not good: 60-70%<br>
We need to reach atleast 85% for reliable stats and worthy paper<br>

Let's attempt at LSTMs for sequence<br>
We may also need attention models on sequences to focus on certain regions of the protein sequence<br>
How about MTL? Using Protein sequence to predict PDB sequence?<br>

### LSTM language model Amino Acids of Protein Sequence

In [10]:
# create mapping of amino acids to integers
aa_to_int = {aa: i for i, aa in enumerate(unique_chars.keys())}
int_to_aa = {i: aa for aa, i in aa_to_int.items()}

print("Vocab size =", len(aa_to_int))

Vocab size = 20


Should window size be fixed or variable? <br>
*** Currently fixing window size to be 20, since minlen is 24

In [24]:
# Prepare the training dataset of input and output pairs encoded as integers
X_train = []
Y_train = []
chain_len = 20 # should we 
for protein in pdb_seq_proteins:
    seq = pdb_sequences[protein]["seq"]
    for i in range(0, len(seq)-chain_len):
        X_train.append([aa_to_int[aa] for aa in seq[i:i+chain_len]])
        Y_train.append(aa_to_int[seq[i+chain_len]])

len(X_train), len(Y_train)

(4332327, 4332327)

In [20]:
"".join([int_to_aa[i] for i in X_train[0]]), int_to_aa[Y_train[0]]

('MGAMTQLLAGVFLAFLALAT', 'E')

In [25]:
import torch 

X_train = torch.tensor(X_train, dtype=torch.int32)
Y_train = torch.tensor(Y_train)

print(X_train.shape, Y_train.shape)

torch.Size([4332327, 20]) torch.Size([4332327])


In [27]:
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [28]:
from torch import nn
from torch.nn import functional as F

class LSTMTextGeneratorChar(nn.Module):
    def __init__(self, n_vocab, embed_len, n_layers, hidden_dim):
        super().__init__()
        self.embed_len = embed_len
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.n_vocab = n_vocab
        self.lstm = nn.LSTM(input_size = embed_len, hidden_size = hidden_dim, num_layers = n_layers, device=device, batch_first = True) ### CAREFUL
        self.dropout = nn.Dropout(p=0.3)
        self.linear = nn.Linear(hidden_dim, n_vocab, device=device)

        self.word_embedding = nn.Embedding(n_vocab, embed_len)

    def forward(self, X_batch):
        embeddings = self.word_embedding(X_batch)

        hidden, carry = torch.randn(self.n_layers, len(X_batch), self.hidden_dim).to(device), torch.randn(self.n_layers, len(X_batch), self.hidden_dim).to(device)
        output, (hidden, carry) = self.lstm(embeddings, (hidden, carry))
        return self.linear(self.dropout(output[:,-1, :]))

In [34]:
import numpy as np
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

In [32]:
from tqdm import tqdm
import copy
def train(model, loss_fn, optimizer, train_loader,epochs=10):
    set_seed(0)

    best_ckpt = None
    min_train_loss = 1000000.0
    for i in range(1, epochs+1):
        losses = []
        print("Current epoch:", i)
        model.train()

        for X_batch, Y_batch in tqdm(train_loader):
            Y_pred = model(X_batch.to(device))
            loss = loss_fn(Y_pred, Y_batch.to(device))
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print("Training loss:", np.mean(losses))
        if np.mean(np.array(losses)) < min_train_loss:
            min_train_loss = np.mean(losses)
            best_ckpt = copy.deepcopy(model)
    return best_ckpt, model

In [36]:
%%time

from torch.optim import Adam

epochs = 10
learning_rate = 5e-3
embed_len = 6
hidden_dim = 128
n_layers=2

set_seed(42)
loss_fn = nn.CrossEntropyLoss().to(device)
text_generator = LSTMTextGeneratorChar(20, embed_len, n_layers, hidden_dim).to(device)
optimizer = Adam(text_generator.parameters(), lr=learning_rate)

CPU times: user 2.58 ms, sys: 11.5 ms, total: 14.1 ms
Wall time: 14.9 ms


In [37]:
best_checkpoint_char, last_checkpoint_char = train(text_generator, loss_fn, optimizer, train_loader, epochs)

Current epoch: 1


  1%|          | 211/16924 [00:10<13:44, 20.26it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/aria/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4522/2537510188.py", line 1, in <module>
    best_checkpoint_char, last_checkpoint_char = train(text_generator, loss_fn, optimizer, train_loader, epochs)
                                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4522/164531175.py", line 13, in train
    for X_batch, Y_batch in tqdm(train_loader):
  File "/home/aria/anaconda3/lib/python3.11/site-packages/tqdm/std.py", line 1178, in __iter__
    for obj in iterable:
  File "/home/aria/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/home/aria/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 676, in _